In [20]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

In [2]:
# 연속된 True 값을 계산하는 함수
def calculate_consecutive_trues(series):
    n = len(series)
    result = [0] * n
    count = 0

    # 첫 번째 패스: 연속된 True의 그룹 길이를 계산
    for i in range(n):
        if series[i]:
            count += 1
        else:
            count = 0
        result[i] = count

    # 두 번째 패스: 그룹의 마지막 True 값에 그룹의 전체 길이를 설정
    final_result = [0] * n
    i = 0
    while i < n:
        if result[i] > 0:
            length = result[i]
            for j in range(length):
                final_result[i - j] = length
            i += length
        else:
            i += 1

    return final_result

In [38]:
def convert_df_to_float(df):
    def try_convert(value):
        try:
            return float(value)
        except (ValueError, TypeError):
            return np.nan
    
    converted_df = df.map(try_convert)
    return converted_df

### 중국의 미세먼지 데이터를 독립변수로 사용하기 위해 처리

- 중국에서 사용할 지점은 베이징, 상하이, 난징, 광저우, 우한, 칭다오, 우루무치 7개 지점
- 각 데이터의 15년 1월 1일 ~ 24년 5월 19일까지의 데이터를 병합한 데이터 생성

In [3]:
# 중국 미세먼지 데이터 호출
beijing_dust = pd.read_csv("./중국 미세먼지 데이터/베이징-air-quality.csv")
shanghai_dust = pd.read_csv("./중국 미세먼지 데이터/상하이-air-quality.csv")
nanjing_dust = pd.read_csv("./중국 미세먼지 데이터/난징-air-quality.csv")
guangzhou_dust = pd.read_csv("./중국 미세먼지 데이터/광저우-air-quality.csv")
wuhan_dust = pd.read_csv("./중국 미세먼지 데이터/우한-air-quality.csv")
qingdao_dust = pd.read_csv("./중국 미세먼지 데이터/칭다오-air-quality.csv")
urumuchi_dust = pd.read_csv("./중국 미세먼지 데이터/우루무치-air-quality.csv")

In [4]:
# 각 지점 데이터의 date 피처 데이터 형식을 날짜로 변경
beijing_dust["date"] = beijing_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))
shanghai_dust["date"] = shanghai_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))
nanjing_dust["date"] = nanjing_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))
guangzhou_dust["date"] = guangzhou_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))
wuhan_dust["date"] = wuhan_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))
qingdao_dust["date"] = qingdao_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))
urumuchi_dust["date"] = urumuchi_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))

In [5]:
# 15년도 이후의 데이터만 남기고 자르기
beijing_dust = beijing_dust[beijing_dust["date"] >= datetime.strptime("2015", "%Y")]
shanghai_dust = shanghai_dust[shanghai_dust["date"] >= datetime.strptime("2015", "%Y")]
nanjing_dust = nanjing_dust[nanjing_dust["date"] >= datetime.strptime("2015", "%Y")]
guangzhou_dust = guangzhou_dust[guangzhou_dust["date"] >= datetime.strptime("2015", "%Y")]
wuhan_dust = wuhan_dust[wuhan_dust["date"] >= datetime.strptime("2015", "%Y")]
qingdao_dust = qingdao_dust[qingdao_dust["date"] >= datetime.strptime("2015", "%Y")]
urumuchi_dust = urumuchi_dust[urumuchi_dust["date"] >= datetime.strptime("2015", "%Y")]

In [6]:
# 데이터 정렬 및 인덱스값 신규 생성
beijing_dust = beijing_dust.sort_values(by = "date").reset_index(drop = True)
shanghai_dust = shanghai_dust.sort_values(by = "date").reset_index(drop = True)
nanjing_dust = nanjing_dust.sort_values(by = "date").reset_index(drop = True)
guangzhou_dust = guangzhou_dust.sort_values(by = "date").reset_index(drop = True)
wuhan_dust = wuhan_dust.sort_values(by = "date").reset_index(drop = True)
qingdao_dust = qingdao_dust.sort_values(by = "date").reset_index(drop = True)
urumuchi_dust = urumuchi_dust.sort_values(by = "date").reset_index(drop = True)

In [7]:
# 각 지점이 어딘지 의미하는 피처 추가
beijing_dust["지점"] = "beijing"
shanghai_dust["지점"] = "shanghai"
nanjing_dust["지점"] = "nanjing"
guangzhou_dust["지점"] = "guangzhou"
wuhan_dust["지점"] = "wuhan"
qingdao_dust["지점"] = "qingdao"
urumuchi_dust["지점"] = "urumuchi"

In [77]:
# 데이터 병합
cn_dust = pd.concat([beijing_dust, shanghai_dust, nanjing_dust, guangzhou_dust, wuhan_dust, qingdao_dust, urumuchi_dust], axis = 0)

In [78]:
# ' pm25', ' pm10'에서 띄어쓰기 제거
cn_dust.columns = cn_dust.columns.str.strip(" ")

In [79]:
# 피봇 적용
cn_dust = cn_dust.pivot(index = "date", columns = "지점", values = ["pm25", "pm10"])

In [80]:
# 24년 5월 20일 이후의 데이터 제거
cn_dust = cn_dust.iloc[:-9, :]

In [81]:
# 결측된 데이터 확인
date_list = pd.date_range(start = "2015-01-01", end = "2024-05-19", freq = "1D")
missing_date = [date for date in date_list if date not in cn_dust.index]

In [82]:
# 결측된 데이터 NaN값으로 채워서 생성
for date in missing_date:
    cn_dust.loc[date, :] = np.nan

In [83]:
# date 기준으로 재정렬
cn_dust = cn_dust.sort_values(by = "date")

In [84]:
# 인덱스 컬럼 명 변경
cn_dust = cn_dust.rename_axis("일시")

In [85]:
# 데이터 타입 float로 변환
cn_dust = convert_df_to_float(cn_dust)

In [86]:
na_count = [cn_dust.loc[date, :].isna().sum() for date in cn_dust.index]
na_4 = [count > 4 for count in na_count]

In [87]:
na_consec = calculate_consecutive_trues(na_4)

In [88]:
pd.Series(na_consec).value_counts()

0    3393
2      18
4      12
1       4
Name: count, dtype: int64

In [46]:
cn_dust.head()

pm25                                                       pm10  \
지점         beijing guangzhou nanjing qingdao shanghai urumuchi  wuhan beijing   
일시                                                                              
2015-01-01   124.0     140.0   133.0     NaN    126.0    173.0  156.0    67.0   
2015-01-02   114.0     136.0   160.0     NaN    163.0    127.0  178.0   145.0   
2015-01-03   210.0     183.0   182.0   155.0    173.0    151.0  222.0   141.0   
2015-01-04   211.0     162.0   195.0   201.0    187.0    143.0  224.0    97.0   
2015-01-05   128.0     161.0   234.0   195.0    158.0    173.0  247.0    45.0   

                                                               
지점         guangzhou nanjing qingdao shanghai urumuchi  wuhan  
일시                                                             
2015-01-01      81.0   105.0     NaN     73.0    109.0  127.0  
2015-01-02     108.0   127.0    69.0     89.0     84.0  181.0  
2015-01-03      71.0   139.0   143.0    100.0     93.0  160.0  
2015-01-04      66.0   175.0   121.0     74.0     96.0  153.0  
2015-01-05      86.0   124.0    43.0     78.0     92.0  140.0

### KNNImputer로 결측치 제거

- 각 데이터의 결측치가 3개 이하인 경우에만 적용

In [48]:
knnimputer = KNNImputer(n_neighbors = 1)

In [49]:
# 스케일링 적용
ss = StandardScaler()
sdf = ss.fit_transform(cn_dust)

In [52]:
# knn으로 결측치 처리 이후 스케일링 복원
knn_df = knnimputer.fit_transform(sdf)
inv_df = ss.inverse_transform(knn_df)
inv_df = pd.DataFrame(inv_df)

In [90]:
# inv_df의 컬럼과 인덱스가 cn_dust와 동일하도록 변경
inv_df.columns = cn_dust.columns
inv_df.index = cn_dust.index

In [92]:
# na_4 값이 False인 경우만 cn_dust의 값 대체
cn_dust.loc[[not i for i in na_4]] = inv_df.loc[[not i for i in na_4]]

### 다른 모든 결측치에 대해서 interpolate(linear) 적용

In [94]:
cn_dust = cn_dust.interpolate("linear")

In [96]:
cn_dust.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3427 entries, 2015-01-01 to 2024-05-19
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (pm25, beijing)    3427 non-null   float64
 1   (pm25, guangzhou)  3427 non-null   float64
 2   (pm25, nanjing)    3427 non-null   float64
 3   (pm25, qingdao)    3427 non-null   float64
 4   (pm25, shanghai)   3427 non-null   float64
 5   (pm25, urumuchi)   3427 non-null   float64
 6   (pm25, wuhan)      3427 non-null   float64
 7   (pm10, beijing)    3427 non-null   float64
 8   (pm10, guangzhou)  3427 non-null   float64
 9   (pm10, nanjing)    3427 non-null   float64
 10  (pm10, qingdao)    3427 non-null   float64
 11  (pm10, shanghai)   3427 non-null   float64
 12  (pm10, urumuchi)   3427 non-null   float64
 13  (pm10, wuhan)      3427 non-null   float64
dtypes: float64(14)
memory usage: 530.6 KB


In [97]:
# 저장 및 다시 읽어들이기
cn_dust.to_csv("./china_dust.csv")
cn_dust = pd.read_csv("./china_dust.csv", header = [0, 1], index_col = 0)
cn_dust.index = cn_dust.index.map(lambda x: datetime.strptime(x, "%Y-%m-%d"))
cn_dust.head()

pm25                                                       pm10  \
지점         beijing guangzhou nanjing qingdao shanghai urumuchi  wuhan beijing   
일시                                                                              
2015-01-01   124.0     140.0   133.0   183.0    126.0    173.0  156.0    67.0   
2015-01-02   114.0     136.0   160.0   148.0    163.0    127.0  178.0   145.0   
2015-01-03   210.0     183.0   182.0   155.0    173.0    151.0  222.0   141.0   
2015-01-04   211.0     162.0   195.0   201.0    187.0    143.0  224.0    97.0   
2015-01-05   128.0     161.0   234.0   195.0    158.0    173.0  247.0    45.0   

                                                               
지점         guangzhou nanjing qingdao shanghai urumuchi  wuhan  
일시                                                             
2015-01-01      81.0   105.0    75.0     73.0    109.0  127.0  
2015-01-02     108.0   127.0    69.0     89.0     84.0  181.0  
2015-01-03      71.0   139.0   143.0    100.0     93.0  160.0  
2015-01-04      66.0   175.0   121.0     74.0     96.0  153.0  
2015-01-05      86.0   124.0    43.0     78.0     92.0  140.0